# [Advent of Code 2020 Day 10](https://adventofcode.com/2020/day/10)

This looks nasty.

## Initial setup

In [ ]:
import ipytest
import sys
sys.path.append("..")
from ansi import *
from comp import *
ipytest.autoconfig()

## Part 1 - A Naive CSP Backtracker
This question has a lot of parallels with CSPs you'll find in AI/CI courses. As such, I'm probably going to try and make a naive backtracking algorithm that incrementally builds the candidate result.

### The Joltage Diff Counter
This will just take an array of nums and multiply the number of 1-diffs by the number of 3-diffs. If there exists a diff outside the 1-3 range it throws.

In [ ]:
def get_product_of_1_and_3_jolt_differences(nums: list[int]) -> int:
    diff_1 = 0
    diff_3 = 0

    for i in range(1, len(nums)):
        if nums[i] - nums[i - 1] == 1:
            diff_1 += 1
        elif nums[i] - nums[i - 1] == 3:
            diff_3 += 1
        elif nums[i] - nums[i - 1] == 2:
            continue
        else:
            raise Exception(f"For array {nums} the diff for {nums[i]=} and {nums[i - 1]=} doesn't satisfy the 1-3 rule")

    return diff_1 * diff_3

In [ ]:
%%ipytest
def test_get_product_of_1_and_3_jolt_differences():
    assert get_product_of_1_and_3_jolt_differences([0, 1, 4, 5, 6, 7, 10, 11, 12, 15, 16, 19, 22]) == 35

### Transform and Conquer
We want every element. The wording of this question implies it always exists. Therefore, we can just sort the input and run the difference function on that.

In [ ]:
def get_multiplied_diffs(nums: list[int]) -> int:
    """
    Takes a list of numbers and orders them so the maximum absolute difference is 3, then returns the number of 1-diffs multiplied by the number of 3-diffs.
    :param nums: numbers to examine
    :return: the product of 1-diffs and 3-diffs
    """
    return get_product_of_1_and_3_jolt_differences([0] + sorted(nums + [max(nums) + 3]))

In [ ]:
%%ipytest
def test_get_multiplied_diffs():
    assert get_multiplied_diffs([16, 10, 15, 5, 1, 11, 7, 19, 6, 12, 4]) == 35

## Main Solver

In [ ]:
def solve(prob, filename):
    lines = []
    gen = yield_line(filename)

    for line in gen:
        lines.append(line)

    nums = list(map(int, lines))

    if prob == 1:
        return get_multiplied_diffs(nums)
    elif prob == 2:
        return 0x3f3f3f3f + 2
    else:
        print("Invalid problem code")
        exit()

In [ ]:
%%ipytest
def test_solve():
    assert solve(1, "example1") == 35
    assert solve(1, "example2") == 220
    assert solve(1, "input") == 2380
    assert solve(2, "input") == 0x3f3f3f3f + 2